The dataset contains transaction data of an online retailer store in Dec, 2010 to Dec, 2011.   
Many customers of the company are wholesalers

Objective: Provide insights to Business Team regarding the sales in the metioned period

[Dataset link](https://www.kaggle.com/datasets/carrie1/ecommerce-data)

In [1]:
from pprint import pprint

from matplotlib import pyplot as plt
from matplotlib.ticker import FuncFormatter
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 100)
import pyarrow as pa
import seaborn as sns

from IPython.display import display, Markdown

In [ ]:
df = pd.read_csv(
	"dataset/uk_retailer.csv", 
	encoding="unicode_escape", 
	date_format="%m/%d/%Y %H:%M", 
	parse_dates=["InvoiceDate"], 
	dtype={
		"InvoiceNo": pd.StringDtype(),
		"StockCode": pd.StringDtype(),
		"Description": pd.StringDtype(),
		"Quantity": pd.Int16Dtype(),
		"UnitPrice": pd.Float64Dtype(),
		"CustomerID": pd.StringDtype(),
		"Country": pd.StringDtype()
	}
)

df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom


In [3]:
df = df.rename(columns={"InvoiceDate": "InvoiceTimestamp"})

df["InvoiceDate"] = df["InvoiceTimestamp"].dt.date
df["InvoiceDate"] = pd.to_datetime(df["InvoiceDate"], format="%Y-%m-%d")

df.dtypes

InvoiceNo           string[python]
StockCode           string[python]
Description         string[python]
Quantity                     Int16
InvoiceTimestamp    datetime64[ns]
UnitPrice                  Float64
CustomerID          string[python]
Country             string[python]
InvoiceDate         datetime64[ns]
dtype: object

In [4]:
# preserve original df
df_ = df.copy()

# Data Cleaning

In [5]:
display(
	df.describe(exclude=[pd.StringDtype()]),
	df.describe(include=[pd.StringDtype()])
)

,Quantity,InvoiceTimestamp,UnitPrice,InvoiceDate
count,541909.0,541909,541909.0,541909
mean,9.55225,2011-07-04 13:34:57.156386048,4.611114,2011-07-04 00:00:13.073782272
min,-15459.0,2010-12-01 08:26:00,-11062.06,2010-12-01 00:00:00
25%,1.0,2011-03-28 11:34:00,1.25,2011-03-28 00:00:00
50%,3.0,2011-07-19 17:17:00,2.08,2011-07-19 00:00:00
75%,10.0,2011-10-19 11:27:00,4.13,2011-10-19 00:00:00
max,15459.0,2011-12-09 12:50:00,38970.0,2011-12-09 00:00:00
std,64.654892,NaN,96.759853,NaN


,InvoiceNo,StockCode,Description,CustomerID,Country
count,541909,541909,540455,406829,541909
unique,25900,4070,4223,4372,38
top,573585,85123A,WHITE HANGING HEART T-LIGHT HOLDER,17841,United Kingdom
freq,1114,2313,2369,7983,495478


Problem:
- Negative quantity and unit price

In [6]:
print("proportion of data come from UK: {}".format(round(df.query("Country.str.lower() == 'united kingdom'").shape[0]/len(df), 2)))

proportion of data come from UK: 0.91


Notes:
- more than 90% of transaction are coming from UK

In [7]:
display(
	"Missing values",
	df.isnull().sum(),
	round(df.isnull().sum()/df.shape[0]*100, 2)
)

'Missing values'

InvoiceNo                0
StockCode                0
Description           1454
Quantity                 0
InvoiceTimestamp         0
UnitPrice                0
CustomerID          135080
Country                  0
InvoiceDate              0
dtype: int64

InvoiceNo            0.00
StockCode            0.00
Description          0.27
Quantity             0.00
InvoiceTimestamp     0.00
UnitPrice            0.00
CustomerID          24.93
Country              0.00
InvoiceDate          0.00
dtype: float64

Problem:
- 0.27% of rows don't have any description
- nearly 25% of transactions were made by unknown customer id

In [8]:
display(
	"unit price = 0",
	df[df["UnitPrice"]==0].head(),
    df[df["UnitPrice"]==0].shape[0],
	df[df["UnitPrice"]==0].shape[0]/len(df)*100
)

'unit price = 0'

,InvoiceNo,StockCode,Description,Quantity,InvoiceTimestamp,UnitPrice,CustomerID,Country,InvoiceDate
622,536414,22139,<NA>,56,2010-12-01 11:52:00,0.0,<NA>,United Kingdom,2010-12-01
1970,536545,21134,<NA>,1,2010-12-01 14:32:00,0.0,<NA>,United Kingdom,2010-12-01
1971,536546,22145,<NA>,1,2010-12-01 14:33:00,0.0,<NA>,United Kingdom,2010-12-01
1972,536547,37509,<NA>,1,2010-12-01 14:33:00,0.0,<NA>,United Kingdom,2010-12-01
1987,536549,85226A,<NA>,1,2010-12-01 14:34:00,0.0,<NA>,United Kingdom,2010-12-01


2515

0.4641000610803659

Problem:
- ~ 0.5% of rows have unit price of 0

In [9]:
invoice_stock_w_multiple_invoice_time = df.groupby(["InvoiceNo", "StockCode"]).agg(
	invoice_time_count = pd.NamedAgg("InvoiceTimestamp", "nunique")
).query("invoice_time_count>=2").sort_values("invoice_time_count", ascending=False).reset_index()

invoice_stock_w_multiple_invoice_time

,InvoiceNo,StockCode,invoice_time_count
0,541596,22748,2
1,541809,21929,2
2,544186,21094,2
3,550320,22839,2
4,567183,22659,2
5,567183,22907,2
6,578548,84347,2


In [10]:
display(
	df.merge(
		invoice_stock_w_multiple_invoice_time,
		how="right",
		on=["InvoiceNo", "StockCode"]
	).groupby(["InvoiceNo", "StockCode"]).agg(
		min_invoice_time = pd.NamedAgg("InvoiceTimestamp", "min"),
		max_invoice_time = pd.NamedAgg("InvoiceTimestamp", "max"),
	).eval(
		"""
		invoice_diff = max_invoice_time - min_invoice_time
		""",
		engine = "python"
	)
)

min_invoice_time    max_invoice_time    invoice_diff
InvoiceNo StockCode                                                        
541596    22748     2011-01-19 16:18:00 2011-01-19 16:19:00 0 days 00:01:00
541809    21929     2011-01-21 14:58:00 2011-01-21 14:59:00 0 days 00:01:00
544186    21094     2011-02-16 15:55:00 2011-02-16 15:56:00 0 days 00:01:00
550320    22839     2011-04-17 12:37:00 2011-04-17 12:38:00 0 days 00:01:00
567183    22659     2011-09-18 15:32:00 2011-09-18 15:33:00 0 days 00:01:00
          22907     2011-09-18 15:32:00 2011-09-18 15:33:00 0 days 00:01:00
578548    84347     2011-11-24 15:02:00 2011-11-24 15:03:00 0 days 00:01:00

In [11]:
display(
	df.merge(
		invoice_stock_w_multiple_invoice_time,
		how="right",
		on=["InvoiceNo", "StockCode"]
	)
)

del invoice_stock_w_multiple_invoice_time

,InvoiceNo,StockCode,Description,Quantity,InvoiceTimestamp,UnitPrice,CustomerID,Country,InvoiceDate,invoice_time_count
0,541596,22748,POPPY'S PLAYHOUSE KITCHEN,1,2011-01-19 16:18:00,2.1,17602,United Kingdom,2011-01-19,2
1,541596,22748,POPPY'S PLAYHOUSE KITCHEN,3,2011-01-19 16:19:00,2.1,17602,United Kingdom,2011-01-19,2
2,541809,21929,JUMBO BAG PINK VINTAGE PAISLEY,2,2011-01-21 14:58:00,4.96,<NA>,United Kingdom,2011-01-21,2
3,541809,21929,JUMBO BAG PINK VINTAGE PAISLEY,2,2011-01-21 14:59:00,4.13,<NA>,United Kingdom,2011-01-21,2
4,544186,21094,SET/6 RED SPOTTY PAPER PLATES,2,2011-02-16 15:55:00,0.83,<NA>,United Kingdom,2011-02-16,2
5,544186,21094,SET/6 RED SPOTTY PAPER PLATES,6,2011-02-16 15:56:00,0.85,<NA>,United Kingdom,2011-02-16,2
6,550320,22839,3 TIER CAKE TIN GREEN AND CREAM,1,2011-04-17 12:37:00,14.95,12748,United Kingdom,2011-04-17,2
7,550320,22839,3 TIER CAKE TIN GREEN AND CREAM,2,2011-04-17 12:38:00,14.95,12748,United Kingdom,2011-04-17,2
8,567183,22659,LUNCH BOX I LOVE LONDON,2,2011-09-18 15:32:00,1.95,14769,United Kingdom,2011-09-18,2
9,567183,22659,LUNCH BOX I LOVE LONDON,2,2011-09-18 15:33:00,1.95,14769,United Kingdom,2011-09-18,2


Problem:
- There are some data grouped by invoice no and stock code have different invoice timestamp
- The invoice timestamp should be exactly the same 
- Interestingly, the difference between highest and lowest invoce timestamp for each group is 1 minute
- and these problems are all happened in UK

In [12]:
odd_stock_code = df.loc[df["StockCode"].str.isalpha(), "StockCode"].unique()
odd_stock_code

<StringArray>
[     'POST',         'D',       'DOT',         'M',         'S', 'AMAZONFEE',
         'm',  'DCGSSBOY', 'DCGSSGIRL',      'PADS',         'B',      'CRUK']
Length: 12, dtype: string

In [ ]:
odd_stock_code_map = {}
for stock_code in odd_stock_code:
	odd_stock_code_map[stock_code] = list(df.loc[df["StockCode"] == stock_code, "Description"].unique())

pprint(odd_stock_code_map)

{'AMAZONFEE': ['AMAZON FEE'],
 'B': ['Adjust bad debt'],
 'CRUK': ['CRUK Commission'],
 'D': ['Discount'],
 'DCGSSBOY': ['BOYS PARTY BAG'],
 'DCGSSGIRL': ['GIRLS PARTY BAG'],
 'DOT': ['DOTCOM POSTAGE', <NA>],
 'M': ['Manual'],
 'PADS': ['PADS TO MATCH ALL CUSHIONS'],
 'POST': ['POSTAGE', <NA>],
 'S': ['SAMPLES'],
 'm': ['Manual']}


Problem:
- stock code (m & M) refers to the same description
- stock D doesn't show any transaction since it's a discount
- does stock code S refering to transaction of samples?
- stock code B doesn't make any sense to be appear in transaction
- only POST and DOT among odd stock codes have 2 product descriptions and one of them is NULL

In [ ]:
df.loc[df["StockCode"]=="S", "Quantity"].value_counts()

Solutions:
- update stock code M to m
- Drop transactions with stock code D, S, B
- update NULL product description of stock code POST and DOT with their non-null description respectively

In [ ]:
for stock_code in sorted(odd_stock_code):
	print(f"stock code: {stock_code}")
	
	for col in ["Quantity", "UnitPrice", "CustomerID", "Country"]:
		print(f"\tunique val in {col}: {df.loc[df['StockCode']==stock_code, col].nunique()}")

		if col in ["Quantity", "UnitPrice"]:
			sns.boxplot(df.query("StockCode==@stock_code"), x=col)
			plt.title(f"Distribution of {col} for Stock Code of {stock_code}")
			plt.show()
		elif col == "Country":
			display(df.loc[df["StockCode"]==stock_code, [col]].value_counts().reset_index().sort_values(by="count", ascending=False))
	
	display(df[df["StockCode"]==stock_code].head(10))

In [ ]:
df[df["UnitPrice"]==0]

**Problem**
- why negative quantity and unitprice present?
- 25% of rows have missing CustomerID
- 0.2% of rows have missing product description
- 2515 observations have unit price of 0

## Indentify the problem

### Negative quantity

In [ ]:
sns.boxplot(
	data = df,
	x = "Quantity"
)

In [ ]:
df[df["Quantity"]<0].shape[0]/df.shape[0]*100

> Only 2% of data contain negative quantity, which very small amount

In [ ]:
df.columns

In [ ]:
df.query("StockCode=='22423' and Quantity<0")["Country"].value_counts()

In [ ]:
df.loc[df["Quantity"]<0, "Country"].value_counts(normalize=True)

- 86% of observation with negative quantity are coming from UK

Resolve:
- Drop rows with negative quantity

### Missing customerid

In [ ]:
df.loc[df["CustomerID"].isnull(), "Country"].value_counts(normalize=True)

In [ ]:
df.query("CustomerID.isnull() and Country=='United Kingdom'").shape[0]/df.query("Country == 'United Kingdom'").shape[0]

In [ ]:
df[df["CustomerID"].isnull()].shape[0]/df.shape[0]

Problem:
- 98% of data with missing CustomerID are coming from UK
- 27% data from UK don't have CustomerID
- 25% of all data don't have CustomerID

In [ ]:
invoiceNo_w_null_CID = df.loc[df["CustomerID"].isnull(), "InvoiceNo"].unique().tolist()
len(invoiceNo_w_null_CID)

In [ ]:
stock_code_w_null_CI = df.query("InvoiceNo.isin(@invoiceNo_w_null_CID)")["StockCode"].value_counts().to_frame("null_count").reset_index()
stock_code_w_null_CI

In [ ]:
df[df["StockCode"].isin(stock_code_w_null_CI.sort_values("null_count", ascending=False).head(10)["StockCode"])].groupby("StockCode")["Description"].unique()

Findings:
- The analysis above used to find out whether there are any pattern on invoice number with emptly customer id
- Interestingly, most of items transacted with condition metioned above are bags

In [ ]:
sns.boxenplot(
    data=stock_code_w_null_CI,
    x="null_count"
)

In [ ]:
df.loc[df["StockCode"].isin(stock_code_w_null_CI["StockCode"]), "StockCode"].value_counts().to_frame("not_null_count").reset_index()

In [ ]:
pd.merge(
	stock_code_w_null_CI,
    (df.loc[
			df["StockCode"]
			.isin(stock_code_w_null_CI["StockCode"]), 
			"StockCode"
        ].value_counts().to_frame("not_null_count")
        .reset_index()
	),
    on="StockCode",
    how="inner"
).eval(
    "null_pct = null_count/not_null_count",
    engine="python"
).sort_values(["null_count", "null_pct"], ascending=False)

In [ ]:
pd.merge(
	stock_code_w_null_CI,
    (df.loc[
			df["StockCode"]
			.isin(stock_code_w_null_CI["StockCode"]), 
			"StockCode"
        ].value_counts().to_frame("not_null_count")
        .reset_index()
	),
    on="StockCode",
    how="inner"
).eval(
    "null_pct = null_count/not_null_count",
    engine="python"
).sort_values(["null_pct", "null_count"], ascending=False)

In [ ]:
df.loc[df["StockCode"]=="47566b"]

Assumptions:
- Customer ID are missing because they weren't registered as a member

Decision:
- Leave observations with null customer id as is

### Duplicated invoice no & stock code

In [ ]:
multiple_invoice_stockCode_pair = (
    df
	.groupby(["InvoiceNo", "StockCode"])
    .size().to_frame("size")
    .sort_values(by="size", ascending=False)
    .query("size>1")
    .reset_index()
)

multiple_invoice_stockCode_pair

In [ ]:
df.query(
    "(InvoiceNo==@multiple_invoice_stockCode_pair.head(1)['InvoiceNo'].values[0])\
	& (StockCode==@multiple_invoice_stockCode_pair.head(1)['StockCode'].values[0])", 
    engine="python"
)

Problem:
- The samples above showing a pair of invoice number and stock code.
- Values across all columns are the same, why these data didn't grouped as one and the quantity summed?
- There are 9,694 pairs of invoice number and stock code infected with the problem

Assumptions:
- These pairs weren't grouped as one due to manual entry or system error

Resolves:
- aggregate these rows into one pair of invoce number and stock code

### Missing product desc

In [ ]:
stock_code_null_desc = df.loc[df["Description"].isnull(), "StockCode"].unique()
stock_code_null_desc

In [ ]:
df.loc[df["StockCode"].isin(stock_code_null_desc), "Country"].value_counts(normalize=True).head(10)

Problems:
- 92% of observations with NULL descriptions are coming from UK

In [ ]:
(df
	.loc[df["StockCode"]
    .isin(stock_code_null_desc), "StockCode"]
    .value_counts(normalize=True)
    .sort_values(ascending=False)
    .head(10)
)

Insights:
- observations with NULL descriptions seem scattered randomly across stock codes

In [ ]:
df_null_desc = (
    df[
		df["StockCode"].isin(stock_code_null_desc)
	]
    .replace({pd.NA: None})
    .groupby("StockCode")["Description"].unique().to_frame("Descriptions")
    .reset_index()
)

display(df_null_desc)
del stock_code_null_desc

In [ ]:
df_null_desc = df_null_desc.loc[
	df_null_desc["Descriptions"].apply(lambda x: None in x and len(x)==2), 
].set_index("StockCode")

df_null_desc

In [ ]:
stock_code_null_desc_filter = df_null_desc.index.to_list()

In [ ]:
def replace_null_desc(desc_list: object):
	"""
	Replace null product description using the following rules:
	
	1. There should be 2 descriptions for the product, one of them is NULL value
	2. Otherwise, the description will be leaved as it is
	
	Param:
		descs (Series): a series that contains each product's description(s)
	Return:
		new series after value replacement
	"""
	desc_list_ = list(desc_list)

	# this statements find non null value in the list
	# I will use if statement to inspect None value, since None means false
	for i in desc_list_:
		if i:
			return i

In [ ]:
df_null_desc["Descriptions"] = df_null_desc["Descriptions"].apply(replace_null_desc)

In [ ]:
null_desc_mapping = df_null_desc["Descriptions"].to_dict()

In [ ]:
mask = df["StockCode"].isin(stock_code_null_desc_filter)

df.loc[mask, "Description"] = df.loc[mask, "StockCode"].map(null_desc_mapping)
del mask

### Unit price of 0

In [ ]:
df[df["UnitPrice"]==0]

In [ ]:
df.loc[df["UnitPrice"]==0, "CustomerID"].isna().sum()

98% of observations with unit price = 0 also have missing value on customer id

In [ ]:
df.query("UnitPrice==0 and CustomerID.isnull()", engine="python")["Country"].value_counts()

99% of observations of unit price = 0 and missing customer id are happened in UK

## Adressing The Problem

### Negative Quantity

In [ ]:
df[df["Quantity"]==0]

In [ ]:
# drop observations with negative quantity
df = df[df["Quantity"]>0]

# Analysis

Business Questions:
- Top 5 Country with most transaction
- Top 5 basket size by country
- Top 5 basket value by country
- MoM sales growth
- What's top 10 product transacted in terms of frequency?
- What's top 5 Countries that generate most revenue?
- Top 5 product with most revenue

In [ ]:
set(df[df.duplicated(subset=["InvoiceNo", "StockCode"])].index.to_list()) - set(df[df.duplicated(subset=["InvoiceNo", "StockCode", "InvoiceTimestamp"])])

In [ ]:
df.loc[598]

In [ ]:
df[df.duplicated(subset=["InvoiceNo", "StockCode", "InvoiceTimestamp"])]

In [ ]:
df[df.duplicated(subset=["InvoiceNo", "StockCode"])]

In [ ]:
df[(df["InvoiceNo"]=="541596") & (df["StockCode"]=="22748")]

In [ ]:
df.groupby(["InvoiceNo", "StockCode"]).agg({"InvoiceTimestamp": "nunique"}).sort_values("InvoiceTimestamp", ascending=False)

In [ ]:
top10_transacted_product = df.groupby("StockCode").agg(
	
)

## Top 5 Countries by Revenue

In [ ]:
top5_country_revenue = df.groupby("Country").agg(
	total_revenue = pd.NamedAgg("revenue", "sum")
).sort_values("total_revenue", ascending=False).reset_index().head()

top5_country_revenue

In [ ]:
fig, ax = plt.subplots(figsize=(6, 3))

sns.barplot(
	data=top5_country_revenue,
	x="total_revenue",
	y="Country",
	ax=ax
)

ax.set_title("Top 5 Country by Revenue")
ax.set_ylabel(None)
ax.set_xlabel(None)
ax.xaxis.set_major_formatter(FuncFormatter(lambda x, _: f"{int(x/1e6)}M"))

ax.spines[["top", "right", "left"]].set_visible(False)
ax.tick_params(
	left = None
)

plt.show()

## Top 5 Transaction by Country

In [ ]:
top5_transaction_country = df.groupby("Country").agg(transaction_num = pd.NamedAgg("InvoiceNo", "nunique")).sort_values(by="transaction_num", ascending=False).head(5).reset_index()
top5_transaction_country["transaction_prop"] = np.round(top5_transaction_country["transaction_num"]/df["InvoiceNo"].nunique(), 3)
top5_transaction_country

In [ ]:
fig, ax = plt.subplots(figsize=(6, 4))
sns.barplot(
	data=top5_transaction_country,
	x="transaction_prop",
	y="Country",
	ax=ax
)

ax.set_title("Top 5 Proportion of Transaction\nby Country", loc="left")
ax.set_ylabel(None)
ax.set_xlabel("transaction proportion")
ax.bar_label(ax.containers[0])

ax.spines[["top", "right", "left"]].set_visible(None)
ax.tick_params(
	left = None
)
plt.show()

> UK has the most transaction ≈90%

## Basket Size

$$
\text{basket size} = \frac{\sum{\text{quantity}}}{\sum{\text{transaction}}}
$$

In [ ]:
df["revenue"] = df["Quantity"]*df["UnitPrice"]
df.head()

In [ ]:
basket_size = df.groupby("Country").agg(
	sum_quantity = pd.NamedAgg("Quantity", "sum"),
	sum_transaction = pd.NamedAgg("InvoiceNo", "nunique"),
).reset_index()

basket_size["basket_size"] = basket_size["sum_quantity"]/basket_size["sum_transaction"]

basket_size = pd.merge(
	left=basket_size,
	right=df.groupby(by=["Country", "InvoiceNo"]).agg(
			total_revenue = pd.NamedAgg("revenue", "sum")
		).reset_index().groupby("Country").agg(
			avg_revenue_per_transaction = pd.NamedAgg("total_revenue", "mean")
		).reset_index()[["Country", "avg_revenue_per_transaction"]],
	how="inner",
	on="Country"
).round({
	"basket_size": 0, 
	"avg_revenue_per_transaction": 0
}).sort_values("basket_size", ascending=False)

In [ ]:
basket_size.head()

- Altough UK has the most proportion of transaction, Netherland generates highest basket size with avg of revenue per transaction is 3 grands

In [ ]:
fig, ax = plt.subplots(figsize=(6, 4))

sns.barplot(
	data=basket_size.head(),
	x="basket_size",
	y="Country",
	ax=ax
)

ax.set_title("Top 5 Basket Size\nby Country", loc="left")
ax.set_ylabel(None)
ax.set_xlabel("Basket Size")

ax.spines[["left", "top", "right"]].set_visible(None)
ax.tick_params(
	left = None
)

plt.show()

## Basket Value

$$
\text{basket value} = \frac{\sum{\text{revenue}}}{\sum{\text{transaction}}}
$$

In [ ]:
basket_value = df.groupby("Country").agg(
	total_revenue = pd.NamedAgg("revenue", "sum"),
	total_transaction = pd.NamedAgg("InvoiceNo", "nunique")
).reset_index()

basket_value["basket_value"] = np.round(
	basket_value["total_revenue"]/basket_value["total_transaction"], 
	0
)
basket_value = basket_value.sort_values(by="basket_value", ascending=False)

basket_value

In [ ]:
fig, ax = plt.subplots(figsize=(6, 4))

sns.barplot(
	data=basket_value.head(),
	y="Country",
	x="basket_value",
	ax=ax
)

ax.set_title("Top 5 Basket Value\nby Country", loc="left")
ax.set_ylabel(None)
ax.set_xlabel("Basket Value")

ax.tick_params(
	left = None
)
ax.spines[["left", "top", "right"]].set_visible(False)

plt.show()

## MoM Sales Growth

$$
\text{MoM Sales Growth} = \frac{\sum{\text{sales of current month}}- \sum{\text{sales of previous month}}}{\sum{\text{sales of previous month}}}
$$

In [ ]:
mom_sales_growth = df.set_index("InvoiceDate").groupby(pd.Grouper(freq="ME")).agg(
	total_sales = pd.NamedAgg("revenue", "sum"),
	total_item_sold = pd.NamedAgg("Quantity", "sum")
)

mom_sales_growth["mom_sales_growth"] = (mom_sales_growth["total_sales"]-mom_sales_growth["total_sales"].shift(periods=1))/mom_sales_growth["total_sales"].shift(periods=1)

mom_sales_growth = mom_sales_growth.reset_index()
mom_sales_growth["period"] = pd.Categorical(
	mom_sales_growth["InvoiceDate"].dt.to_period("M").dt.strftime("%Y-%m"),
	categories=['2010-12', '2011-01', '2011-02', 
				'2011-03', '2011-04', '2011-05', 
				'2011-06', '2011-07', '2011-08', 
				'2011-09', '2011-10', '2011-11', 
				'2011-12'],
	ordered=True
)
mom_sales_growth

In [ ]:
mom_sales_growth2 = mom_sales_growth.copy()
mom_sales_growth2["period"] = mom_sales_growth.dropna()["period"].cat.remove_unused_categories()
fig, ax = plt.subplots(figsize=(10, 4))
sns.lineplot(
	data=mom_sales_growth2,
	x="period",
	y="mom_sales_growth",
	ax=ax
)

ax.set_xticks(ax.get_xticks())
ax.set_xticklabels(ax.get_xticklabels(), rotation=45)

ax.set_title("MoM Sales Growth")
ax.set_ylabel(None)
ax.set_xlabel(None)

ax.spines[["top", "right"]].set_visible(False)

plt.show()

del mom_sales_growth2

In [ ]:
fig, ax = plt.subplots(figsize=(10, 4))

sns.lineplot(
	data=mom_sales_growth,
	x="period",
	y="total_item_sold",
	ax=ax
)

ax.set_title("Sum of Quantity Sold over the Period")
ax.set_xlabel(None)
ax.set_ylabel(None)

ax.set_ybound(0, 700000)
ax.set_xticks(ax.get_xticks())
ax.set_xticklabels(ax.get_xticklabels(), rotation=45)

plt.show()

> 

In [ ]:
df["InvoiceTimestamp"].dt.to_period("M").dt.strftime("%Y-%m")

In [ ]:
df["Country"].unique().tolist()

In [ ]:
revenue_df = df.groupby(by=["InvoiceDate", "Country"]).agg(total_revenue = pd.NamedAgg("revenue", "sum")).reset_index()
revenue_df.head()

In [ ]:
sns.lineplot(
	data = df.groupby("InvoiceDate").agg(total_revenue = pd.NamedAgg("revenue", "sum")),
	x = "InvoiceDate",
	y = "total_revenue"
)